In [1]:
import os
import pandas as pd
from github import Github
from credentials import username, password
from data_API import dataAPI

In [2]:
g = Github(username(), password())  # credentials for github login
repos = g.get_user().get_repos()  # iterable; all the repos associated with the profile incl. organizations

In [3]:
api_data = dataAPI(repos)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


## Aggregate Data

In [4]:
df_data = pd.DataFrame(api_data)
all_unique_pairs = df_data['Pair'].unique()

### 1. Make dataset with commit size - additions & deletions

In [9]:
grouped_data = df_data.groupby(by=['Pair','Date','Author']).describe()

x = grouped_data['Additions'][['max']].reset_index()
x['Deletions'] = grouped_data['Deletions'][['max']].reset_index()['max']
x = x.rename(columns={'max': 'Additions'})
grouped_data = x

In [10]:
aggregated_list = []

for pair in all_unique_pairs:

    df_pair = grouped_data[grouped_data['Pair'] == pair]

    members = df_pair['Author'].unique()

    if len(members) == 2: # best scenario

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]
            m_1 = df_pair_date[df_pair_date['Author'] == members[0]]
            m_2 = df_pair_date[df_pair_date['Author'] == members[1]]

            m1_additions = df_pair_date[df_pair_date['Author'] == members[0]]['Additions'].sum()
            m2_additions = df_pair_date[df_pair_date['Author'] == members[1]]['Additions'].sum()

            m1_deletions = df_pair_date[df_pair_date['Author'] == members[0]]['Deletions'].sum()
            m2_deletions = df_pair_date[df_pair_date['Author'] == members[0]]['Deletions'].sum()


            aggregated_list.append([pair, date, m_1, m_2, m1_additions, m2_additions, m1_deletions, m2_deletions])

    elif len(members) > 2: 

        
        print('\nIssue! More than 2 pair members with pair', pair)
        print('Members: ', members)

    elif len(members) < 2: 
        print('\nIssue! Less than 2 pair members with pair', pair)
        print('Members: ', members)


Issue! More than 2 pair members with pair 11
Members:  ['justinalittlefield' 'AER-Stud' 'kajiewang']

Issue! More than 2 pair members with pair 18
Members:  ['Steefkuh' 'diederikpel' 'dtpel']

Issue! Less than 2 pair members with pair 19
Members:  ['Chri5thiano']

Issue! More than 2 pair members with pair 8
Members:  ['invalid-email-address' 'willemsjob' 'RobHoogma']


In [11]:
cols = ['Pair', 'Date', 'Member_1', 'Member_2', 'M1_Additions', 'M2_Additions', 'M1_deletions', 'M2_deletions']
df_aggregated_member = pd.DataFrame(aggregated_list, columns=cols).sort_values(by='Pair')
df_aggregated_member['Total'] = df_aggregated_member['Member_1'] + df_aggregated_member['Member_2']

### 2. Make Dataset with total of additions + deletions

In [13]:
df_total_size = df_aggregated_member[['Pair', 'Date']]
df_total_size['Member_1'] = df_aggregated_member['M1_Additions'] + df_aggregated_member['M1_deletions']
df_total_size['Member_2'] = df_aggregated_member['M2_Additions'] + df_aggregated_member['M2_deletions']
df_total_size['Total'] = df_total_size['Member_1'] + df_total_size['Member_2'] 

### 3. Make dataset with commits

In [16]:
grouped_data = df_data.groupby(by=['Pair','Date','Author']).describe()
grouped_data = grouped_data['Additions'][['count']].reset_index()

In [23]:
aggregated_list = []

for pair in all_unique_pairs:

    df_pair = grouped_data[grouped_data['Pair'] == pair]

    members = df_pair['Author'].unique()

    if len(members) == 2: # best scenario

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]
            m_1 = df_pair_date[df_pair_date['Author'] == members[0]]['count'].sum()
            m_2 = df_pair_date[df_pair_date['Author'] == members[1]]['count'].sum()

            aggregated_list.append([pair, date, m_1, m_2])

    elif len(members) > 2:

        custom_more = []

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]
            m_1 = df_pair_date[df_pair_date['Author'] == members[0]]['count'].sum()
            m_2 = df_pair_date[df_pair_date['Author'] == members[1]]['count'].sum()
            m_2 += df_pair_date[df_pair_date['Author'] == members[2]]['count'].sum()

            custom_more.append([pair, date, m_1, m_2])

        aggregated_list += custom_more 

    elif len(members) < 2:

        custom_less = []

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]
            m_1 = df_pair_date[df_pair_date['Author'] == members[0]]['count'].sum()
            m_2 = 0

            custom_less.append([pair, date, m_1, m_2])

        aggregated_list += custom_less

In [24]:
cols = ['Pair', 'Date', 'Member_1', 'Member_2']
df_commits = pd.DataFrame(aggregated_list, columns=cols).sort_values(by='Pair')
df_commits['Total'] = df_commits['Member_1'] + df_commits['Member_2']

In [25]:
df_commits

,Pair,Date,Member_1,Member_2,Total
84,2,2020-04-30,1.0,0.0,1.0
91,2,2020-05-14,3.0,4.0,7.0
89,2,2020-05-06,4.0,0.0,4.0
88,2,2020-05-05,0.0,2.0,2.0
90,2,2020-05-13,4.0,5.0,9.0
...,...,...,...,...,...
370,89,2020-05-07,9.0,3.0,12.0
369,89,2020-05-06,2.0,0.0,2.0
373,89,2020-05-18,1.0,0.0,1.0
382,90,2020-05-05,6.0,3.0,9.0
